In [1]:
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField

from openff.interchange.components.interchange import Interchange

In [2]:
# Create an OpenFF Topology consisting of two ethanol molecules
top = Topology.from_molecules(2 * [Molecule.from_smiles("CCO")])

In [3]:
# Load in two versions of OpenFF 1.x.x ("Parsley")
parsley_100 = ForceField("openff-1.0.0.offxml")
parsley_130 = ForceField("openff-1.3.0.offxml")

In [4]:
# Construct an Interchange object from each force field and the common topology
sys_100 = Interchange.from_smirnoff(force_field=parsley_100, topology=top)
sys_130 = Interchange.from_smirnoff(force_field=parsley_130, topology=top)

In [5]:
# Look into each object's angle values ...
sys_100.handlers["Angles"].potentials

{PotentialKey(id='[*:1]~[#6X4:2]-[*:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(101.737336, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(107.660782, 'degree')>}, map_key=None),
 PotentialKey(id='[*:1]-[#8:2]-[*:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(112.364889, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(110.25178, 'degree')>}, map_key=None),
 PotentialKey(id='[#1:1]-[#6X4:2]-[#1:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(74.2870153, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(107.599151, 'degree')>}, map_key=None)}

In [6]:
# ... and notice that they're (slightly) different values
sys_130.handlers["Angles"].potentials

{PotentialKey(id='[*:1]~[#6X4:2]-[*:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(99.2339941, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(113.65694, 'degree')>}, map_key=None),
 PotentialKey(id='[*:1]-[#8:2]-[*:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(134.501978, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(110.289839, 'degree')>}, map_key=None),
 PotentialKey(id='[#1:1]-[#6X4:2]-[#1:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(66.5522943, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(114.294085, 'degree')>}, map_key=None)}

In [7]:
# This can be verified by directly comparing the objects
assert not sys_100.handlers["Angles"] == sys_130.handlers["Angles"]

In [8]:
# (This first step not strictly necessary, as the typing did not change between versions of this force field line)
sys_100.handlers["Angles"].store_matches(parsley_100["Angles"], topology=top)

# But we can replace the `Potential` objects with openff-1.3.0 values by feeding it in
# the "Bonds" section of that force field, which is represented in memory by a `BondHandler` object
sys_100.handlers["Angles"].store_potentials(parsley_130["Angles"])

In [9]:
assert sys_100.handlers["Angles"] == sys_130.handlers["Angles"]

In [10]:
# Or, more verbosely, we can again inspect the objects themselves ...
sys_100.handlers["Angles"].potentials

{PotentialKey(id='[*:1]~[#6X4:2]-[*:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(99.2339941, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(113.65694, 'degree')>}, map_key=None),
 PotentialKey(id='[*:1]-[#8:2]-[*:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(134.501978, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(110.289839, 'degree')>}, map_key=None),
 PotentialKey(id='[#1:1]-[#6X4:2]-[#1:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(66.5522943, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(114.294085, 'degree')>}, map_key=None)}

In [11]:
# ... and see that they are now both using the `openff-1.3.0` values
sys_130.handlers["Angles"].potentials

{PotentialKey(id='[*:1]~[#6X4:2]-[*:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(99.2339941, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(113.65694, 'degree')>}, map_key=None),
 PotentialKey(id='[*:1]-[#8:2]-[*:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(134.501978, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(110.289839, 'degree')>}, map_key=None),
 PotentialKey(id='[#1:1]-[#6X4:2]-[#1:3]', mult=None, associated_handler='Angles', bond_order=None): Potential(parameters={'k': <Quantity(66.5522943, 'kilocalorie / mole / radian ** 2')>, 'angle': <Quantity(114.294085, 'degree')>}, map_key=None)}

In [12]:
# But we didn't change any other values, i.e. bonds
assert not sys_100.handlers["Bonds"] == sys_130.handlers["Bonds"]

In [13]:
# TODO: Add a trip to disk